In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("spark_sql") \
    .config("spark.jars", "/Users/viktoria/Documents/apache_spark/postgresql-42.5.3.jar") \
    .getOrCreate()

23/06/09 19:05:02 WARN Utils: Your hostname, MacBook-Air-Viktoria.local resolves to a loopback address: 127.0.0.1; using 192.168.0.4 instead (on interface en0)
23/06/09 19:05:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/06/09 19:05:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [23]:
# reading data using JDBC drivers

df_part_1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/test_data") \
    .option("dbtable", "data_part_1") \
    .option("user", "postgres") \
    .option("password", "0880") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_part_2 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/test_data") \
    .option("dbtable", "data_part_2") \
    .option("user", "postgres") \
    .option("password", "0880") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_part_3 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/test_data") \
    .option("dbtable", "data_part_3") \
    .option("user", "postgres") \
    .option("password", "0880") \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [25]:
# join tables into one table
df = df_part_1.union(df_part_2).union(df_part_3)

In [26]:
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- county: string (nullable = true)
 |-- postal: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- email: string (nullable = true)



In [22]:
from pyspark.sql.functions import expr, count, sum, col

In [27]:
# calculating fraction of domains

df = df.withColumn("domain", expr("substring_index(email, '@', -1)"))
total_count = df.count()
domain_counts = df.groupBy("domain").agg(count("*").alias("domain_count"))
domain_percentage = domain_counts.withColumn("domain_ratio", (col("domain_count") / total_count) * 100)

domain_percentage.show(domain_percentage.count())

+-----------------+------------+------------------+
|           domain|domain_count|      domain_ratio|
+-----------------+------------+------------------+
|    teplica.co.uk|           1|0.3105590062111801|
|        gmail.com|          83| 25.77639751552795|
|      brace.co.uk|           1|0.3105590062111801|
|     torner.co.uk|           1|0.3105590062111801|
|      drawe.co.uk|           1|0.3105590062111801|
|      yadao.co.uk|           1|0.3105590062111801|
|     tauras.co.uk|           1|0.3105590062111801|
|      rampy.co.uk|           1|0.3105590062111801|
|  gastellum.co.uk|           1|0.3105590062111801|
|      ungar.co.uk|           1|0.3105590062111801|
|        yahoo.com|          80| 24.84472049689441|
|     bassil.co.uk|           1|0.3105590062111801|
|      hotmail.com|          67| 20.80745341614907|
|    paletta.co.uk|           1|0.3105590062111801|
|     kufner.co.uk|           1|0.3105590062111801|
|stifflemire.co.uk|           1|0.3105590062111801|
|   mancilla

In [28]:
# calculating count of people for each company 
people_count = df.groupBy("company_name").agg(count("*").alias("people_count"))
people_count.show(people_count.count(), truncate=False)

+-----------------------------+------------+
|company_name                 |people_count|
+-----------------------------+------------+
|Wilhelm, James E Jr          |53          |
|Practical Periphrals         |12          |
|Bomarko Inc                  |23          |
|Automation Engrg & Mfg Inc   |1           |
|Guynes Design Inc            |1           |
|Storm Eye Institute          |1           |
|Lutz, Christopher T Esq      |21          |
|company_name                 |1           |
|Alan D Rosenburg Cpa Pc      |62          |
|Ernest E Heuer & Company Pc  |2           |
|Cavuto, John A               |26          |
|Chile Pepper Magazine        |1           |
|Flash, Elena Salerno Esq     |1           |
|M D & W Railway              |12          |
|Eagle Newspapers             |14          |
|Reliable Yard Service        |1           |
|Levy Security Consultants Ltd|9           |
|Owensboro Brick & Tile       |21          |
|Chemical Consultants         |15          |
|Don Allen